In [49]:
import numpy as np
import pandas as pd
import json
import datetime as dt
import os

path_to_json = "../data/binance/futures/1d"


In [50]:
json_files = [json for json in os.listdir(path_to_json)]


In [51]:
js_dfs = []
for json_name in json_files:
    temp_json = json.load(open(f"{path_to_json}/{json_name}"))
    temp_df = pd.DataFrame(temp_json)
    temp_df['noise'] = 1- abs((temp_df.loc[:,1] - temp_df.loc[:,4])/
                                    (temp_df.loc[:,2] - temp_df.loc[:,3]))
        
    temp_df["noise_3d_average"] = temp_df["noise"].rolling(3).mean()
    temp_df["noise_5d_average"] = temp_df["noise"].rolling(5).mean()
    temp_df["date"] = pd.to_datetime(temp_df.loc[:, 0], unit ='ms')
    temp_df["date"]= temp_df["date"].apply(str)
    temp_df["date"]= temp_df["date"].apply(lambda x: x[:-9])
    temp_df["name"] = json_name.split('_')[0]
    js_dfs.append(temp_df)
len(js_dfs)

146

In [52]:
js_dfs[3].head()

,0,1,2,3,4,5,noise,noise_3d_average,noise_5d_average,date,name
0,1608854400000,2.7000,2.9562,1.7498,2.2801,71519996.0,0.651940,NaN,NaN,2020-12-25,1INCH
1,1608940800000,2.2823,2.4260,1.5595,1.5960,72874458.0,0.207963,NaN,NaN,2020-12-26,1INCH
2,1609027200000,1.5960,1.6540,1.0415,1.0636,96158019.0,0.130776,0.330226,NaN,2020-12-27,1INCH
3,1609113600000,1.0615,1.2800,1.0409,1.1175,52486917.0,0.765788,0.368176,NaN,2020-12-28,1INCH
4,1609200000000,1.1172,1.1204,0.7563,0.8866,95372584.0,0.366658,0.421074,0.424625,2020-12-29,1INCH


In [53]:
months=[]
for i in range(1,13):
    if i <10:
        months.append("0" + str(i) )
    else:
        months.append(str(i))
years = ["2020","2021","2022",]
dates = ["2019-10-01","2019-11-01","2019-12-01"]
for year in years:
    for month in months:
        dates.append(year + "-" + month + '-' +"01" )
print(dates)

['2019-10-01', '2019-11-01', '2019-12-01', '2020-01-01', '2020-02-01', '2020-03-01', '2020-04-01', '2020-05-01', '2020-06-01', '2020-07-01', '2020-08-01', '2020-09-01', '2020-10-01', '2020-11-01', '2020-12-01', '2021-01-01', '2021-02-01', '2021-03-01', '2021-04-01', '2021-05-01', '2021-06-01', '2021-07-01', '2021-08-01', '2021-09-01', '2021-10-01', '2021-11-01', '2021-12-01', '2022-01-01', '2022-02-01', '2022-03-01', '2022-04-01', '2022-05-01', '2022-06-01', '2022-07-01', '2022-08-01', '2022-09-01', '2022-10-01', '2022-11-01', '2022-12-01']


In [54]:
ranking = pd.DataFrame(0, index=np.arange(len(js_dfs)),columns=dates)
print(ranking)


     2019-10-01  2019-11-01  2019-12-01  2020-01-01  2020-02-01  2020-03-01  \
0             0           0           0           0           0           0   
1             0           0           0           0           0           0   
2             0           0           0           0           0           0   
3             0           0           0           0           0           0   
4             0           0           0           0           0           0   
..          ...         ...         ...         ...         ...         ...   
141           0           0           0           0           0           0   
142           0           0           0           0           0           0   
143           0           0           0           0           0           0   
144           0           0           0           0           0           0   
145           0           0           0           0           0           0   

     2020-04-01  2020-05-01  2020-06-01  2020-07-01

In [55]:
print(js_dfs[0]['date'].isin(["2022-09-09"]))
print("2022-09-09" in js_dfs[0]['date'].values)
print(js_dfs[0]['date'])

0       True
1      False
2      False
3      False
4      False
       ...  
115    False
116    False
117    False
118    False
119    False
Name: date, Length: 120, dtype: bool
True
0      2022-09-09
1      2022-09-10
2      2022-09-11
3      2022-09-12
4      2022-09-13
          ...    
115    2023-01-02
116    2023-01-03
117    2023-01-04
118    2023-01-05
119    2023-01-06
Name: date, Length: 120, dtype: object


In [78]:
for date in dates:
    # print(date)
    temp_list = []
    for df in js_dfs:
        df['noise_5d_average'].fillna(1, inplace=True)
        if(date in df['date'].values ):
            # temp_list.append(df.loc[df['date'] == date, df['noise_5d_average']]) 
            # temp_list.concat(df.loc[df['date'] == date,['noise_5d_average']], df.loc[df['date'] == date,['name']])
            # print(df.loc[df['date'] == date,['noise_5d_average']].values[0][0])
            temp_list.append((df.loc[df['date'] == date,['noise_5d_average']].values[0][0], df.loc[df['date'] == date,['name']].values[0][0]))
    # print(temp_list)
    if(len(temp_list) <len(js_dfs)):
        for i in range(len(js_dfs) - len(temp_list)):
            temp_list.append((1, "empty"))
        
    ranking[date] = temp_list

In [79]:
ranking = ranking.apply(lambda x: sorted(x, key = lambda x:x[0]))
ranking.to_json("../data/binance/monthly_pairs_by_5d_noise.json")


0      (0.37704996094453225, KAVA)
1       (0.4292382999377531, REEF)
2       (0.43470297405477254, OMG)
3       (0.43819588505769486, SNX)
4       (0.4496104041045113, UNFI)
                  ...             
141                     (1, empty)
142                     (1, empty)
143                     (1, empty)
144                     (1, empty)
145                     (1, empty)
Name: 2021-04-01, Length: 146, dtype: object


In [82]:
ranking_top5 = ranking.iloc[:5,:]
ranking_top5.to_json("../data/binance/monthly_pairs_by_5d_noise_top5.json")